# -------------------- Day 12 of 100 days SQL Challenge ------------

In [1]:
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')


# Establish a connection to the SQLite database
db_path = "FinalDB.db"  # Replace with the correct path if not in the same directory
connection = sqlite3.connect(db_path)

In [2]:
%load_ext sql



In [3]:

%sql sqlite:///FinalDB.db


In [4]:

# path for the data
delivery_partner_path = "delivery_partner.csv"
food_path = "food.csv"
menu_path = 'menu.csv'
order_detail_path = 'order_details.csv'
order_path = 'orders (1).csv'
restaurant_path='restaurants.csv'
users_path = 'users.csv'



# Load data into Pandas DataFrames from the provided path
delivery_partner_df = pd.read_csv(delivery_partner_path)
food_df = pd.read_csv(food_path)
menu_df = pd.read_csv(menu_path)
order_detail_df = pd.read_csv(order_detail_path)
order_df = pd.read_csv(order_path)
restaurant_df=pd.read_csv(restaurant_path)
users_df = pd.read_csv(users_path)

# Use the to_sql method to create tables and insert DataFrames into SQLite tables

delivery_partner_df.to_sql('delivery_partner', connection, index=False, if_exists='replace')
food_df.to_sql('food', connection, index=False, if_exists='replace')
menu_df.to_sql('menu', connection, index=False, if_exists='replace')
order_detail_df.to_sql('order_detail', connection, index=False, if_exists='replace')
order_df.to_sql('order', connection, index=False, if_exists='replace')
restaurant_df.to_sql('restaurant', connection, index=False, if_exists='replace')
users_df.to_sql('users', connection, index=False, if_exists='replace')


7

In [5]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';



 * sqlite:///FinalDB.db
Done.


name
loyal_users
delivery_partner
food
menu
order_detail
order
restaurant
users


## Data dictionary  - 

### delivery_partner - 

1. **partner_id (int):** An integer representing the unique identifier for each partner.
2. **partner_name (str):** A string representing the name of the partner.

### food - 

1. **f_id (int):** An integer representing the unique identifier for each food item.
2. **f_name (str):** A string representing the name of the food item.
3. **type (str):** A string representing the type of the food item, indicating whether it's "Veg" or "Non-veg".

### menu - 

1. **menu_id (int):** An integer representing the unique identifier for each menu item.
2. **r_id (int):** An integer representing the restaurant ID indicating which restaurant the menu item belongs to.
3. **f_id (int):** An integer representing the food item ID indicating which food item is included in the menu.
4. **price (int):** An integer representing the price of the menu item.

### order_detail - 

1. **id (int):** An integer representing the unique identifier for each record.
2. **order_id (int):** An integer representing the identifier for each order.
3. **f_id (int):** An integer representing the food item ID indicating which food item is part of the order.

### order -

1. **order_id (int):** Identifier for each order.
2. **user_id (int):** Identifier for each user.
3. **r_id (int):** Identifier for each restaurant.
4. **amount (int):** The total amount for the order.
5. **date (date):** The date when the order was placed.
6. **partner_id (int):** Identifier for each partner.
7. **delivery_time (int):** Time taken for delivery in minutes.
8. **delivery_rating (int):** Rating given for the delivery service (out of 5).
9. **restaurant_rating (int):** Rating given for the restaurant (out of 5).

### restaurant -

1. **r_id (int):** Identifier for each restaurant.
2. **r_name (str):** Name of the restaurant.
3. **cuisine (str):** Type of cuisine offered by the restaurant.


### users - 

1. **user_id (int):** Identifier for each user.
2. **name (str):** Name of the user.
3. **email (str):** Email address of the user.
4. **password (str):** Password associated with the user's account.





#### problem 1 -
find all users who have never ordered


In [6]:
%%sql
SELECT *
FROM USERS
WHERE user_id NOT IN (SELECT DISTINCT user_id FROM "ORDER")

 * sqlite:///FinalDB.db
Done.


user_id,name,email,password
6,Anupama,anupama@gmail.com,46rdw2
7,Rishabh,rishabh@gmail.com,4sw123


#### PROBLEM 2- 
find the favourite food of each customer

In [7]:
%%sql
with fav_food as (
    SELECT o.user_id,name,f_name, COUNT(*) AS `frequency`
    FROM users u
JOIN "order" o ON u.user_id=o.user_id
JOIN order_detail od ON o.order_id=od.order_id
JOIN food f ON od.f_id = f.f_id
GROUP BY o.user_id,od.f_id
)

SELECT * FROM fav_food f1
WHERE frequency = (SELECT MAX(frequency)
                  FROM fav_food f2
                  WHERE f2.user_id = f1.user_id)
ORDER BY frequency DESC


 * sqlite:///FinalDB.db
Done.


user_id,name,f_name,frequency
1,Nitish,Choco Lava cake,5
5,Neha,Choco Lava cake,5
2,Khushboo,Choco Lava cake,3
3,Vartika,Chicken Wings,3
4,Ankit,Schezwan Noodles,3
4,Ankit,Veg Manchurian,3


#### PROBLEM 3- 
Display average rating of all the restaurants

In [8]:
%%sql
select r_name, avg_rating
from (select r_id, avg(restaurant_rating) as `avg_rating`
     from 'order'
     group by r_id) t1 JOIN restaurant t2
     on t1.r_id = t2.r_id 

 * sqlite:///FinalDB.db
Done.


r_name,avg_rating
dominos,1.6666666666666667
kfc,2.2
box8,4.666666666666667
Dosa Plaza,3.6666666666666665
China Town,3.6666666666666665


In [9]:
%%sql
-- Drop the table if it exists
DROP TABLE IF EXISTS loyal_users;

-- Create the table
CREATE TABLE loyal_users (
    user_id INTEGER PRIMARY KEY,
    name VARCHAR(30),
    money INTEGER
);


 * sqlite:///FinalDB.db
Done.
Done.


[]

In [10]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';



 * sqlite:///FinalDB.db
Done.


name
delivery_partner
food
menu
order_detail
order
restaurant
users
loyal_users


#### Problem - 
Populate a already created loyal_users table with records only those customers who have ordered food more than 3 times.

In [11]:
%%sql
INSERT INTO loyal_users (user_id, name)
SELECT t1.user_id, t2.name
FROM "order" t1
JOIN users t2 ON t1.user_id = t2.user_id
GROUP BY t1.user_id, t2.name
HAVING count(*) > 3;


 * sqlite:///FinalDB.db
5 rows affected.


[]

In [12]:
%%sql
select * from loyal_users

 * sqlite:///FinalDB.db
Done.


user_id,name,money
1,Nitish,None
2,Khushboo,None
3,Vartika,None
4,Ankit,None
5,Neha,None


#### Problem - 
populate the money col of loyal_users table using the order table. provide 10% app money to all customer based on thier order value

In [13]:
%%sql
update loyal_users
set money = (select sum(amount)*0.1
            from 'order' as o
            where o.user_id=loyal_users.user_id)

 * sqlite:///FinalDB.db
5 rows affected.


[]

In [14]:
%%sql
select * from loyal_users

 * sqlite:///FinalDB.db
Done.


user_id,name,money
1,Nitish,166.5
2,Khushboo,267
3,Vartika,132
4,Ankit,180
5,Neha,303.5


#### Problem
Delete all the customer record who have never ordered

In [15]:
%%sql
delete from users where user_id in
(select user_id from users
where user_id not in (select distinct(user_id) from 'order'))

 * sqlite:///FinalDB.db
2 rows affected.


[]